This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to rotation splines](index.ipynb)

# Cumulative Form

The basic idea, as proposed by
<cite data-cite="kim1995general">Kim, Kim and Shin (1995)</cite>
(section 4) is the following:

Instead of representing a curve as a sum of basis functions
weighted by its control point's position vectors $p_i$
(as it's for example done with [Bézier splines](../euclidean/bezier.ipynb)),
they suggest to use the relative difference vectors $\Delta p_i$ between successive control points.

These relative difference vectors can then be "translated" to *local* rotations
(replacing additions with multiplications),
leading to a form of rotation splines.

## Piecewise Slerp

As an example,
they define a piecewise linear curve

\begin{equation*}
p(t) =
p_0 +
\sum_{i=1}^n \alpha_i(t) \Delta p_i,
\end{equation*}

where

\begin{align*}
\Delta p_i &= p_i - p_{i - 1}\\
\alpha_i(t) &= \begin{cases}
0 & t < i - 1\\
t - i + 1 & i - 1 \leq t < i\\
1 & t \geq i.
\end{cases}
\end{align*}

In [ ]:
def alpha(i, t):
    if t < i - 1:
        return 0
    elif t >= i:
        return 1
    else:
        return t - i + 1

<div class="alert alert-info">

Note

There is an off-by-one error in the paper's definition of $\alpha_i(t)$:

\begin{equation*}
\alpha_i(t) = \begin{cases}
0 & t < i\\
t - i & i \leq t < i + 1\\
1 & t \geq i + 1.
\end{cases}
\end{equation*}

This assumes that $i$ starts with $0$,
but it actually starts with $1$.

</div>

This "cumulative form" can be "translated" to a rotation spline
by replacing addition with multiplication
and the relative difference vectors by relative (i.e. local) rotations
(represented by unit quaternions):

\begin{equation*}
q(t) =
q_0
\prod_{i = 1}^n \exp(\omega_i \alpha_i(t)),
\end{equation*}

where

\begin{equation*}
\omega_i =
\log\left(q_{i - 1}^{-1} q_i\right).
\end{equation*}

The paper uses above notation,
but this could equivalently be written as

\begin{equation*}
q(t) =
q_0
\prod_{i = 1}^n \left(q_{i - 1}^{-1} q_i\right)^{\alpha_i(t)}.
\end{equation*}

In [ ]:
import numpy as np

[helper.py](helper.py)

In [ ]:
from helper import angles2quat, animate_rotations, display_animation

In [ ]:
from splines.quaternion import UnitQuaternion

In [ ]:
# NB: math.prod() since Python 3.8
product = np.multiply.reduce

In [ ]:
def piecewise_slerp(qs, t):
    return qs[0] * product([
        (qs[i - 1].inverse() * qs[i])**alpha(i, t)
        for i in range(1, len(qs))])

In [ ]:
qs = [
    angles2quat(0, 0, 0),
    angles2quat(90, 0, 0),
    angles2quat(90, 90, 0),
    angles2quat(90, 90, 90),
]

In [ ]:
times = np.linspace(0, len(qs) - 1, 100)

In [ ]:
ani = animate_rotations([piecewise_slerp(qs, t) for t in times], figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## Cumulative Bézier/Bernstein Curve

After the piecewise Slerp,
<cite data-cite="kim1995general">Kim, Kim and Shin (1995)</cite>
show (in section 5.1) how to create a *cumulative form*
inspired by Bézier splines, i.e. using Bernstein polynomials.

They start with the well-known equation for Bézier splines:

\begin{equation*}
p(t) =
\sum_{i=0}^n p_i \beta_{i,n}(t),
\end{equation*}

where $\beta_{i,n}(t)$ are Bernstein basis functions as shown in
[the notebook about Bézier splines](../euclidean/bezier-de-casteljau.ipynb#Arbitrary-Degree).

They re-formulate this into a *cumulative form*:

\begin{equation*}
p(t) =
p_0 \tilde{\beta}_{0,n}(t) +
\sum_{i=1}^n \Delta p_i \tilde{\beta}_{i,n}(t),
\end{equation*}

where the cumulative Bernstein basis functions are given by

\begin{equation*}
\tilde{\beta}_{i,n}(t) =
\sum_{j=i}^n \beta_{j,n}(t).
\end{equation*}

We can get the Bernstein basis polynomials via the function
[splines.Bernstein.basis()](../python-module/splines.rst#splines.Bernstein.basis):

In [ ]:
from splines import Bernstein

... and create a simple helper function to sum them up:

In [ ]:
from itertools import accumulate

In [ ]:
def cumulative_bases(degree, t):
    return list(accumulate(Bernstein.basis(degree, t)[::-1]))[::-1]

Finally, they "translate" this into a rotation spline using quaternions, like before:

\begin{equation*}
q(t) =
q_0
\prod_{i=1}^n \exp\left(\omega_i \tilde{\beta}_{i,n}(t)\right),
\end{equation*}

where

\begin{equation*}
\omega_i =
\log(q_{i-1}^{-1} q_i).
\end{equation*}

Again, they use above notation in the paper,
but this could equivalently be written as

\begin{equation*}
q(t) =
q_0
\prod_{i=1}^n \left(q_{i-1}^{-1} q_i\right)^{\tilde{\beta}_{i,n}(t)}.
\end{equation*}

In [ ]:
def cumulative_bezier(qs, t):
    degree = len(qs) - 1
    bases = cumulative_bases(degree, t)
    assert np.isclose(bases[0], 1)
    return qs[0] * product([
        (qs[i - 1].inverse() * qs[i])**bases[i]
        for i in range(1, len(qs))
    ])

In [ ]:
times = np.linspace(0, 1, 100)

In [ ]:
rotations = [cumulative_bezier(qs, t) for t in times]

In [ ]:
ani = animate_rotations(rotations, figsize=(4, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

## Comparison with De Casteljau's Algorithm

> This Bézier quaternion curve has a different
> shape from the Bézier quaternion curve
> of <cite data-cite="shoemake1985animating">Shoemake (1985)</cite>. 
>
> --<cite data-cite="kim1995general">Kim, Kim and Shin (1995)</cite>, section 5.1

The method described by <cite data-cite="shoemake1985animating">Shoemake (1985)</cite>
is shown in [a separate notebook](de-casteljau.ipynb).
An implementation is available in the
[splines.quaternion.DeCasteljau](../python-module/splines.quaternion.rst#splines.quaternion.DeCasteljau) class:

In [ ]:
from splines.quaternion import DeCasteljau

In [ ]:
times = np.linspace(0, 1, 100)

In [ ]:
control_polygon = [
    angles2quat(90, 0, 0),
    angles2quat(0, -45, 90),
    angles2quat(0, 0, 0),
    angles2quat(180, 0, 180),
]

In [ ]:
cumulative_rotations = [
    cumulative_bezier(control_polygon, t)
    for t in times
]

In [ ]:
cumulative_rotations_reversed = [
    cumulative_bezier(control_polygon[::-1], t)
    for t in times
][::-1]

In [ ]:
casteljau_rotations = DeCasteljau([control_polygon]).evaluate(times)

In [ ]:
ani = animate_rotations({
    'De Casteljau': casteljau_rotations,
    'Cumulative': cumulative_rotations,
    'Cumulative reversed': cumulative_rotations_reversed,
}, figsize=(9, 3))

In [ ]:
display_animation(ani, default_mode='reflect')

Applying the same method on the reversed list of control points
and then time-reversing the resulting sequence of rotations
leads to an equal (except for rounding errors) sequence of rotations
when using De Casteljau's algorithm:

In [ ]:
casteljau_rotations_reversed = DeCasteljau([control_polygon[::-1]]).evaluate(times)[::-1]

In [ ]:
for one, two in zip(casteljau_rotations, casteljau_rotations_reversed):
    assert np.isclose(one.scalar, two.scalar)
    assert np.isclose(one.vector[0], two.vector[0])
    assert np.isclose(one.vector[1], two.vector[1])
    assert np.isclose(one.vector[2], two.vector[2])

However, doing the same thing with the "cumulative form"
can lead to a significantly different sequence,
as can be seen in the above animation.